# 手写 word2vec 模型和训练

word2vec 不是一种单一算法，而是一系列模型架构和优化，可用于从大型数据集中学习单词嵌入。通过 word2vec 学习的嵌入已被证明在各种下游自然语言处理任务上是成功的。

Note: 本教程基于 [Efficient estimation of word representations in vector space](https://arxiv.org/pdf/1301.3781.pdf) 和 [Distributed representations of words and phrases and their compositionality](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)。 这不是论文的具体实施。相反，它旨在说明关键思想。

这些论文提出了两种学习单词表示的方法：

*   **Continuous bag-of-words model**: 基于周围上下文词预测中间词。上下文由当前（中间）单词前后的几个单词组成。这种体系结构被称为单词包模型，因为上下文中单词的顺序并不重要。

*   **Continuous skip-gram model**: 预测同一句子中当前单词前后一定范围内的单词。下面给出了一个工作示例。

在本教程中，您将使用 skip-gram 方法。首先，您将使用一个句子来说明跳过语法和其他概念。接下来，您将在一个小数据集上训练自己的word2vec模型。本教程还包含导出经过训练的嵌入并在中可视化它们的代码 [TensorFlow Embedding Projector](http://projector.tensorflow.org/).


## Skip-gram 和负采样 (negative sampling)

当 bag-of-words 模型预测给定相邻上下文的单词时，skip-gram 模型预测给定单词本身的单词上下文（或相邻）。该模型基于跳过图进行训练，skip-gram 是允许跳过 tokens 的 n-grams（参见下图）。一个词的上下文可以通过一组 “(target_word，context_words)” 的跳过语法对来表示，其中 “context_ word” 出现在 “target-word” 的相邻上下文中。

考虑以下八个单词的句子：

> The wide road shimmered in the hot sun.

该句子的 8 个单词中的每一个的上下文单词由窗口大小定义。窗口大小确定可被视为“上下文词”的 “target_word” 两侧的词的跨度。下面是基于不同窗口大小的目标词的 skip-grams。

Note: 对于本教程，窗口大小 “n” 表示每侧有 n 个单词，整个窗口跨度为 2*n+1 个单词。

![word2vec_skipgrams](https://tensorflow.org/tutorials/text/images/word2vec_skipgram.png)

skip-gram 模型的训练目标是最大化预测给定目标词的上下文词的概率。对于一系列单词 *w<sub>1</sub>、w<sub>2</sub>、…w<sub>T</sub>*，目标可以写成平均对数概率。

![word2vec_skipgram_objective](https://tensorflow.org/tutorials/text/images/word2vec_skipgram_objective.png)

其中 ‘c’ 是训练上下文的大小。基本跳跃图公式使用 softmax 函数定义该概率。

![word2vec_full_softmax](https://tensorflow.org/tutorials/text/images/word2vec_full_softmax.png)

其中，*v* 和 *v<sup>'<sup>* 是单词的目标和上下文向量表示，*W* 是词汇大小。

计算该公式的分母涉及对整个词汇表单词执行完整的 softmax，这些单词通常是大的（10<sup>5</sup>-10<supp>7</supp>）。

[noise contrastive estimation](https://www.tensorflow.org/api_docs/python/tf/nn/nce_loss) (NCE) 损失函数是完整softmax的有效近似。为了学习单词嵌入而不是建模单词分布，NCE 损失可以使用负采样方法 [simplified](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) 

目标词的简化负采样目标是将上下文词与从词的噪声分布 *P<sub>n</sub>（w）* 中提取的 “num_ns” 负样本区分开来。更准确地说，词汇表上的完整 softmax 的有效近似是，对于跳转语法对，将目标词的丢失作为上下文词和 “num_ns” 负样本之间的分类问题。

负样本被定义为 `（target_word，context_words）` 对，使得 ` context_ word'不出现在` target_ word'的 ` window_size` 邻域中。对于示例语句，这是几个潜在的负样本（当 ‘window_size’ 为 ‘2’ 时）。

```
(hot, shimmered)
(wide, hot)
(wide, sun)
```

在下一节中，您将为单个句子生成 skip-gram 和负样本。在本教程的后面部分，您还将学习子采样技术，并为正和负训练示例训练分类模型。

## 安装包

In [ ]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

### 矢量化例句

考虑以下句子：

> The wide road shimmered in the hot sun.

给句子分词:

In [ ]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

创建词汇表以保存从令牌到整数索引的映射：

In [ ]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)

创建反向词汇表以保存从整数索引到令牌的映射：

In [ ]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

矢量化你的句子：

In [ ]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

### 从一个句子生成 skip-grams 

`tf.keras.preprocessing` “序列”模块提供了有用的功能，简化了 word2vec 的数据准备。您可以使用 `tf.keras.preprocessing.sequence`。skip-grams 用于从具有给定 ‘window_size’ 的 ‘example_sequence’ 中从范围 `[0，vocab_size）` 中的标记生成跳过图对。

Note: 在这里，`negative_samples` 设置为 0，因为批处理此函数生成的负样本需要单独的代码。在下一节中，您将使用另一个函数执行负采样。

In [ ]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)
print(len(positive_skip_grams))

打印几个 skip-grams：

In [ ]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

### skip-gram 的负采样

`skipgrams` 函数通过在给定的窗口跨度上滑动来返回所有正的 skip-grams 对。要生成额外的跳过语法对，用作训练的负样本，您需要从词汇表中随机抽取单词。使用 `tf.random`。log_uniform_candidate_sampler 函数对窗口中给定目标字的负样本数进行采样。您可以在一个 skip-grams 的目标词上调用该函数，并将上下文词作为 true 类传递，以将其从采样中排除。

关键点: 对于小的数据集 `num_ns` (the number of negative samples per a positive context word) 的范围在 `[5, 20]`  [shown to work](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) ， 对于的大的数据集 `num_ns` 的范围在 `[2, 5]`。

In [ ]:
# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

### 构建一个训练样本示例 

For a given positive `(target_word, context_word)` skip-gram, you now also have `num_ns` negative sampled context words that do not appear in the window size neighborhood of `target_word`. Batch the `1` positive `context_word` and `num_ns` negative context words into one tensor. This produces a set of positive skip-grams (labeled as `1`) and negative samples (labeled as `0`) for each target word

对于给定的正 `（target_word，context_words）` skip-gram，您现在也有 `num_ns` 负采样的上下文字，它们不出现在 `target_ word` 的窗口大小附近。将 1 设置为正 context_word 和 num_ns 负上下文词批处理为一个张量。这为每个目标字产生一组正跳转图（标记为“1”）和负样本（标记为”0”）。

In [ ]:
# Add a dimension so you can use concatenation (in the next step).
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")

# Reshape the target to shape `(1,)` and context and label to `(num_ns+1,)`.
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

从上面的 skip-gram 示例中查看目标词的上下文和相应标签：

In [ ]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

一个由`（target，context，label）` 张量组成的元组构成了一个训练示例，用于训练跳过革兰氏阴性采样word2vec模型。请注意，目标的形状为 `（1，）`，而上下文和标签的 shape 为 `（1+num_ns，）`

In [ ]:
print("target  :", target)
print("context :", context)
print("label   :", label)

### 总结

此图总结了从句子生成训练示例的过程：

![word2vec_negative_sampling](https://tensorflow.org/tutorials/text/images/word2vec_negative_sampling.png)

注意，`temperature` 和 `code` 不是输入句子的一部分。它们与上图中使用的某些其他索引一样属于词汇表。

## 将所有步骤融合到一个函数


### Skip-gram 采样表

大数据集意味着词汇表更大，更频繁的词（如停止词）数量更多。通过对常见单词（如“the”、“is”、“on”）进行采样而获得的训练示例并没有为模型提供多少有用的信息。[Mikolov et al.](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) 建议对频繁词进行二次采样，以提高嵌入质量。

`tf.keras.preprocessing.sequence.skipgrams` 函数接受一个采样表参数，以编码对任何令牌进行采样的概率。您可以使用`tf.keras.preprocessing.sequence`。make_sampling_table 生成基于词频秩的概率采样表，并将其传递给 skipgrams 函数。检查 vocab_size 为 10 的采样概率。

In [ ]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

`sampling_table[i]` 表示对数据集中第i个最常用单词进行采样的概率。该函数基于字频率采样。 [Zipf's distribution](https://en.wikipedia.org/wiki/Zipf%27s_law)

关键点: `tf.random.log_uniform_candidate_sampler` 已经假设词汇频率遵循对数均匀（Zipf）分布。使用这些分布加权采样也有助于使用更简单的损失函数来近似噪声对比估计（NCE）损失，以训练负采样目标。

### 生成训练数据

将上述所有步骤编译成一个函数，该函数可以在从任何文本数据集获得的矢量化语句列表上调用。请注意，采样表是在对跳过字对进行采样之前构建的。您将在后面的章节中使用此函数。

In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

## 为 word2vec 准备训练数据

了解了如何使用一个基于 skip-gram 负采样的 word2vec 模型的句子，您可以继续从更大的句子列表中生成训练示例！

### 下载文本语料库

本教程将使用莎士比亚作品的文本文件。更改以下行以在您自己的数据上运行此代码。

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

从文件中读取文本并打印前几行：

In [ ]:
with open(path_to_file) as f:
  lines = f.read().splitlines()
for line in lines[:20]:
  print(line)

使用非空行构造`tf.data.TextLineDataset`对象，用于下一步：

In [ ]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

### 从语料库中矢量化句子

你可以使用 `TextVectorization` 把 corpus 中的句子向量化。 关于 `TextVectorization` 可以从 [Text classification](https://www.tensorflow.org/tutorials/keras/text_classification) 得到更多的细节. 请注意，从上面的前几句话中，文本需要在一种情况下，标点符号需要删除。为了做到这个，我们需要定义 `custom_standardization` 函数， 可以在文本矢量化层中使用。

In [ ]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

调用 `TextVectorization.adapt` 在文本数据集上使用 “adapt ”创建词汇表。

In [ ]:
vectorize_layer.adapt(text_ds.batch(1024))

一旦该层的状态被调整为表示文本语料库，就可以使用 `TextVectorization.get_vocabulary` 获取词典。 此函数返回按频率排序（降序）的所有词汇表标记的列表。

In [ ]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

`vectorize_layer` 可以对于每一个元素生成向量 `text_ds` (`tf.data.Dataset`)。并可以应用 `Dataset.batch`, `Dataset.prefetch`, `Dataset.map` 和 `Dataset.unbatch`.

In [ ]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

### 从数据集中获取序列


您现在有了一个 `tf.data.Dataset`。整数编码句子的数据集。要准备用于训练 word2vec 模型的数据集，请将数据集展平为句子向量序列列表。这一步是必需的，因为您将迭代数据集中的每个句子，以生成正负样本。

Note:  `generate_training_data()` 在前面定义的使用非TensorFlow Python/NumPy函数时，您还可以使用 `tf.py_function` 或 `tf.numpy_function` 和 `tf.data.Dataset.map`.

In [ ]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

查看以下示例 `sequences`:

In [ ]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

### 从序列生成训练示例

`sequences` 现在是int编码语句的列表。只需调用前面定义的 `generate_training_data` 函数即可为 word2vec 模型生成训练示例。总而言之，该函数迭代每个序列中的每个单词，以收集正面和负面上下文单词。目标、上下文和标签的长度应相同，表示训练示例的总数。

In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")


### 为性能配置数据集

要对可能大量的训练示例执行有效的批处理，请使用 `tf.data.Dataset` API. 在这一步之后， `tf.data.Dataset` 中包含 `(target_word, context_word), (label)`  元素去训练 word2vec 模型!

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

应用 `Dataset.cache` 和 `Dataset.prefetch` 去提高性能:

In [ ]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

## 模型和训练

word2vec 模型可以被实现为分类器，以区分来自跳过图的真实上下文词和通过负采样获得的虚假上下文词。您可以在目标词和上下文词的嵌入之间执行点积乘法，以获得标签预测，并根据数据集中的真实标签计算损失函数。

### word2vec 模型类

使用 [Keras Subclassing API](https://www.tensorflow.org/guide/keras/custom_layers_and_models) 去定义你的 to define your word2vec 模型:

* `target_embedding`: `tf.keras.layers.Embedding` 层, 我们可以从中去查找目标单词的 embedding，这一层的超参数量为 `(vocab_size * embedding_dim)`

* `context_embedding`: 另外 `tf.keras.layers.Embedding` 层, 当一个词作为上下文词出现时，查找该词的嵌入。 该层中的参数数量与 `target_embedding`, 比如 `(vocab_size * embedding_dim)` 中的相同

* `dots`: `tf.keras.layers.Dot` 从训练对计算目标和上下文嵌入的点积的层。

* `flatten`: `tf.keras.layers.Flatten` layer 将 `dots` 层的结果展平为 Logit。 

使用子类模型，您可以定义接受 `（target，context）` 对的 ` call（） `函数，然后将其传递到相应的嵌入层。重塑 “context_embedded” 以执行与“target_embodded” 的点积，并返回展平后的结果。

关键点: “target_embedded” 和 “context_embudded” 层也可以共享。您还可以使用两个嵌入的串联作为最终的 word2vec 嵌入。

In [ ]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

### 定义损失函数并编译模型


为了简单起见，您可以使用 `tf.keras.loss.CategoricalCrossEntropy` 作为负采样损失的替代方案。如果您想编写自己的自定义损耗函数，也可以按如下方式编写：

``` python
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)
```

是时候建立你的模型了！使用128的嵌入维度实例化word2vec类（您可以使用不同的值进行实验）。使用 `tf.keras.optimizers.Adam` 优化器编译模型。

In [ ]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

还定义一个回调来记录 TensorBoard 的训练统计信息：

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

在 `dataset` 上对模型进行若干 epochs 的训练：

In [ ]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

TensorBoard 现在显示 word2vec 模型的精度和损失：

In [ ]:
#docs_infra: no_execute
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="images/word2vec_tensorboard.png"/> -->

## 词嵌入查找和分析

Obtain the weights from the model using `Model.get_layer` and `Layer.get_weights`. The `TextVectorization.get_vocabulary` function provides the vocabulary to build a metadata file with one token per line.

使用 `Layer.get_weights` 和 `Model.get_layer` 从模型中获取权重。`TextVectorization.get_vocabulary` 函数提供词汇表的获取。

In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

创建并保存矢量和元数据文件：

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

下载 `vectors.tsv` 和 `metadata.tsv` 去在 [Embedding Projector](https://projector.tensorflow.org/) 中可视化分析:

In [ ]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass